In [0]:
import etl
import pandas as pd
import numpy as np

In [0]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, cohen_kappa_score
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler

In [0]:
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from lightgbm import LGBMClassifier

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from scikeras.wrappers import KerasClassifier
from keras.utils import to_categorical

In [0]:
%%time
df = etl.load(files=["data/US_Accidents_March23.csv"])
df.shape

In [0]:
df.head()

## Model Selection

In [0]:
def stratified_downsample(df, target_col, downsample_pct=0.2):
    _, downsampled_df = train_test_split(df, test_size=downsample_pct, stratify=df[target_col], random_state=42)
    return downsampled_df

In [0]:
def plot_confusion_matrix(cm):
    plt.figure(figsize=(4, 3))
    sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues", cbar=False, xticklabels=[1, 2, 3, 4], yticklabels=[1, 2, 3, 4])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.show()

In [0]:
def evaluate(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    kappa = cohen_kappa_score(y_true, y_pred)
    print(f"Accuracy: {accuracy:.3f}, F1 Score: {f1:.3f}, Cohen's Kappa: {kappa:.3f}")

In [0]:
def plot_feat_importance(data):
    importances, feature_names = data
    # Plotting the feature importances
    plt.figure(figsize=(8, 5))
    plt.barh(feature_names, importances)
    plt.title("Feature Importance")
    plt.xlabel("Importance")
    plt.ylabel("Feature")
    plt.tight_layout()
    plt.show()

In [0]:
def train_pipeline(clf, build_fn=None, data=None, files=None, ignore_vars=None, ordinal_feats=[], test_size=0.3, 
                   scoring='f1_weighted', grid_search=False, params=None, sampling_params=None, 
                   show_cm=True, include_importance=False, full_train=False, device='cpu'):
    # 1-Load Data
    if data is None and files is None:
        raise Exception("DataFrame or list of files expected.")
    elif files is not None:
        print("Importing data...")
        data = etl.load(files)
    
    # 2-Define preprocessing steps
    # Define features and target
    if "Severity" not in data.columns:
        raise ValueError("Column 'Severity' not found in the data.")
    
    X = data.drop(columns=["Severity"])
    y = data["Severity"]
    
    # Ignore certain columns
    if ignore_vars is None:
        ignore_vars = []
        
    X = X[[c for c in X.columns if c not in ignore_vars]]
    
    # Normalize numerical variables
    num_features = X.select_dtypes(include=['number']).columns.tolist()
    num_transformer = StandardScaler() if num_features else 'passthrough'
    
    # One-hot encode categorical variables
    cat_features = [c for c in X.select_dtypes(include=['object', 'category']).columns.tolist() if c not in ordinal_feats]
    cat_transformer = OneHotEncoder(drop='first', handle_unknown='ignore') if cat_features else 'passthrough'
    
    # Handle ordinal variables
    ord_features = [c for c in ordinal_feats if c in X.columns]
    ord_transformer = 'passthrough'
    
    # Handle boolean variables
    bool_features = X.select_dtypes(include=['bool']).columns.tolist()
    bool_transformer = 'passthrough'
    
    # Encode ordinal target
    target_transformer = OrdinalEncoder(categories=[[1, 2, 3, 4]])
    y = target_transformer.fit_transform(y.values.reshape(-1, 1)).ravel()  # Flatten the target array
    if clf == "mlp":
        n_classes = len(np.unique(y))
        # One-hot encode the target variable
        y = to_categorical(y, num_classes=n_classes)
    
    if sampling_params:
        print("Resampling with SMOTE..")
        # Add SMOTE oversampling to the pipeline
        smote_method = sampling_params.get('smote-method')
        sampling_strategy = sampling_params.get('sampling_strategy', 'auto')
        
        if smote_method:
            if type(sampling_strategy) is dict:
                sampling_strategy = {k: int(np.ceil(v*(1-test_size)*len(X))) for k,v in sampling_strategy.items()}
        
            if smote_method == 'class-wise':
                smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)
            elif smote_method == 'borderline':
                smote = BorderlineSMOTE(sampling_strategy=sampling_strategy, random_state=42)
            else:
                raise Exception(f"SMOTE method '{smote_method}' not supported")
        else:
            smote = 'passthrough'
        
        # Add Downsampling to the pipeline
        downsampling_strategy = sampling_params.get('downsampling_strategy')
        
        if downsampling_strategy:
            if type(downsampling_strategy) is dict:
                downsampling_strategy = {k: int(np.floor(v*(1-test_size)*len(X))) for k,v in downsampling_strategy.items()}
        
            downsampler = RandomUnderSampler(sampling_strategy=downsampling_strategy, random_state=42)
        else:
            downsampler = 'passthrough'
    else:
        smote = 'passthrough'
        downsampler = 'passthrough'
    
    # Create preprocessing pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', num_transformer, num_features),
            ('cat', cat_transformer, cat_features),
            ('ord', ord_transformer, ord_features),
            ('bool', bool_transformer, bool_features),
        ])
    
    # 3-Split data into train-test set
    if full_train:
        X_train = X
        y_train = y
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, stratify=y, random_state=42)
    
    if clf == "mlp":
        # Fit the preprocessor on the training data to transform the features
        preprocessor.fit(X_train)
        X_train_transformed = preprocessor.transform(X_train)
        # Define MLP classifier
        model = KerasClassifier(model=build_fn, 
                              input_dim=X_train_transformed.shape[1],
                              output_dim=n_classes,
                              hid_dim=[64, 64],
                              learning_rate=0.001,
                              epochs=10, 
                              batch_size=32, 
                              dropout_rate=0.3, 
                              verbose=0)
    
    # 4-Train model
    model_pipeline = ImbPipeline(steps=[
        ('preprocessor', preprocessor),
        ('smote', smote),
        ('downsampling', downsampler),
        ('clf', model)
    ])
    
    if grid_search and params:
        # Optional hyperparameter tuning
        grid_search_cv = GridSearchCV(model_pipeline, params, cv=5, scoring=scoring)
        print("Training model with GridSearchCV..")
        grid_search_cv.fit(X_train, y_train)
        # Use the best model from GridSearchCV
        model = grid_search_cv.best_estimator_
        print("Training completed..")
        # Print best hyperparameters
        print("Best hyperparameters:\n", grid_search_cv.best_params_)
    else:
        print("Training model..")
        model_pipeline.fit(X_train, y_train)
        model = model_pipeline
        print("Training completed..")
        
    # 5-Test model
    
    print("Model evaluation..")
    print("\nTrain metrics:")
    y_train_pred = model.predict(X_train)
    if clf == "mlp":
        y_train_pred = np.argmax(y_train_pred, axis=1)
        y_train = np.argmax(y_train, axis=1)
    print(clf == "mlp")
    evaluate(y_train, y_train_pred)
    
    if not full_train:
        print("\nTest metrics:")
        y_pred = model.predict(X_test)
        if clf == "mlp":
            y_pred = np.argmax(y_pred, axis=1)
            y_test = np.argmax(y_test, axis=1)
        evaluate(y_test, y_pred)
    
        print("\nClassification report:")
        print(classification_report(y_test, y_pred))
    
        if show_cm:
            print("\nConfusion matrix:")
            cm = confusion_matrix(y_test, y_pred, normalize='true')
            plot_confusion_matrix(cm)
    
        print("Evaluation completed..")
    
    # 6-Optional plot feature importance
    if include_importance and hasattr(model.named_steps['clf'], 'feature_importances_'):
        feature_names = num_features + bool_features + ord_features
        # Access the OneHotEncoder to get the feature names after fitting
        if isinstance(cat_transformer, OneHotEncoder):
            cat_feature_names = model.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(cat_features)
            feature_names += list(cat_feature_names)   
        importances = model.named_steps['clf'].feature_importances_

        if include_importance and importances is not None:
            return model, (importances, feature_names)
    return model

### Decision Tree (Baseline)

In [0]:
dt_params = {
    'clf__criterion': ['entropy'],#['gini', 'entropy'],
    'clf__max_depth': [None, 10, 20, 30, 50],
    'clf__min_samples_split': [2, 5], #[2, 5, 10, 20],
    'clf__min_samples_leaf': [1, 5, 10, 15],
    'clf__max_features': [None, 'sqrt'],
    'clf__splitter': ['best', 'random'],
    'clf__random_state': [42],
    'clf__class_weight': ['balanced']
}

In [0]:
%%time
_, feat_importance_dt0 = train_pipeline(clf=DecisionTreeClassifier(), data=df, ignore_vars=None, ordinal_feats=["Month"], 
                               test_size=0.3, scoring='f1_weighted', 
                               grid_search=True, params=dt_params,
                               show_cm=True, include_importance=True)

In [0]:
plot_feat_importance(feat_importance_dt0)

### XGBoost

In [0]:
xgb_params = {
    'clf__n_estimators': [100, 150, 200],
    'clf__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'clf__max_depth': [3, 4, 5, 6, 10, 15],
    'clf__subsample': [0.6, 0.8, 1.0],
}

In [0]:
xgb_clf = xgb.XGBClassifier(device='cuda', tree_method='hist', use_label_encoder=False)

In [0]:
%%time
_, feat_importance_xgb0 = train_pipeline(clf=xgb_clf, data=df, ignore_vars=None, ordinal_feats=["Month"], 
                                        test_size=0.3, scoring='f1_weighted', grid_search=True, params=xgb_params, 
                                        sampling_params=None, show_cm=True, include_importance=True)

In [0]:
plot_feat_importance(feat_importance_xgb0)

#### Resampling
Considering the severe class imbalance in the dataset, we use SMOTE to oversample the minority classes. We may consider two approaches: Class-wise SMOTE and Borderline-SMOTE.

*Class-wise SMOTE:* Independently oversamples each minority class to a specified level. It is flexible and simple for multiclass imbalances but ay introduce noise by generating synthetic samples in overlapping regions.

In [0]:
# Class-wise SMOTE
sampling_params_xgb = {
    "smote-method": "class-wise",
    "sampling_strategy": {0: 0.15, 3: 0.15},
    "downsampling_strategy": {1: 0.5}
}

In [0]:
%%time
_, feat_importance_xgb1 = train_pipeline(clf=xgb_clf, data=df, ignore_vars=None, ordinal_feats=["Month"], 
                                        test_size=0.3, scoring='f1_weighted', grid_search=True, params=xgb_params, 
                                        sampling_params=sampling_params_xgb, show_cm=True, include_importance=True)

*Borderline-SMOTE:* Focuses on oversampling near the decision boundary to improve classification in difficult areas. It improves performance by targeting challenging instances but also relies on identifying borderline instances, which can be complex.

In [0]:
# Borderline-SMOTE
sampling_params_xgb = {
    "smote-method": "borderline",
    "sampling_strategy": {0: 0.15, 3: 0.15},
    "downsampling_strategy": {1: 0.5}
}

In [0]:
%%time
_, feat_importance_xgb2 = train_pipeline(clf=xgb_clf, data=df, ignore_vars=None, ordinal_feats=["Month"], 
                                        test_size=0.3, scoring='f1_weighted', grid_search=True, params=xgb_params, 
                                        sampling_params=sampling_params_xgb, show_cm=True, include_importance=True)

In [0]:
plot_feat_importance(feat_importance_xgb2)

### LightGBM

In [0]:
lgbm_params = {
    'clf__n_estimators': [100, 150, 200],
    'clf__learning_rate': [0.01, 0.05, 0.1],
    'clf__min_child_samples': [10, 20, 30],
    'clf__min_split_gain': [0.1, 0.2],
    'clf__subsample': [1.0],
    'clf__colsample_bytree': [0.6, 0.8, 1.0],
    'clf__num_leaves': [50, 100, 150],
}

In [0]:
# Borderline-SMOTE
sampling_params_lgbm = {
    "smote-method": "borderline",
    "sampling_strategy": {0: 0.15, 3: 0.15},
    "downsampling_strategy": {1: 0.5}
}

In [0]:
lgbm_clf = LGBMClassifier(device_type='gpu', verbose=-1)

In [0]:
%%time
_, feat_importance_lgbm = train_pipeline(clf=lgbm_clf, data=df, ignore_vars=None, ordinal_feats=["Month"], 
                                        test_size=0.3, scoring='f1_weighted', grid_search=True, params=lgbm_params, 
                                        sampling_params=sampling_params_lgbm, show_cm=True, include_importance=True)

### Neural Network (MLP)

In [0]:
def create_mlp_model(input_dim, output_dim, learning_rate=0.001, hid_dim=[128, 64], dropout_rate=0.3):
    model = Sequential()
    # Input layer and first hidden layer
    model.add(Input(shape=(input_dim,)))  # Keep this as the input layer
    model.add(Dense(hid_dim[0], activation='relu'))
    model.add(Dropout(dropout_rate))
    # Second hidden layer
    model.add(Dense(hid_dim[1], activation='relu'))
    model.add(Dropout(dropout_rate))
    # Output layer (assuming multi-class classification)
    model.add(Dense(output_dim, activation='softmax'))
    
    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # Use built-in accuracy instead of f1
    return model

In [0]:
mlp_params = {
    'clf__hid_dim': [[256, 128]],#[[128, 64], [128, 128], [256, 128], [256, 256]],
    'clf__dropout_rate': [0.2],#[0.2, 0.4, 0.6],
    'clf__learning_rate': [0.0001],#[0.0001, 0.001, 0.01],
    'clf__batch_size': [16],#[16, 32],
    'clf__epochs': [100],
}

In [0]:
# Borderline-SMOTE
sampling_params_mlp = {
    "smote-method": "borderline",
    "sampling_strategy": {0: 0.15, 3: 0.15},
    "downsampling_strategy": {1: 0.5}
}

In [0]:
%%time
_ = train_pipeline(clf="mlp", build_fn=create_mlp_model, data=df, ignore_vars=None, ordinal_feats=["Month"], 
                                        test_size=0.3, scoring='f1_weighted', grid_search=True, params=mlp_params,
                                        sampling_params=sampling_params_mlp, show_cm=True, include_importance=True)

In [0]:
%%time
_ = train_pipeline(clf="mlp", build_fn=create_mlp_model, data=df, ignore_vars=None, ordinal_feats=["Month"], 
                                        test_size=0.3, scoring='f1_weighted', grid_search=True, params=mlp_params,
                                        sampling_params=sampling_params_mlp, show_cm=True, include_importance=True)

## Train best model with full dataset

In [0]:
from datetime import datetime
import joblib

In [0]:
params = {
    'learning_rate': 0.1, 
    'max_depth': 10, 
    'n_estimators': 200, 
    'subsample': 0.8, 
    'use_label_encoder': False, 
    'device': 'cuda', 
    'tree_method': 'hist'
}

In [0]:
sampling_params = {
    "smote-method": "borderline",
    "sampling_strategy": {0: 0.15, 3: 0.15},
    "downsampling_strategy": {1: 0.5}
}

In [0]:
clf = xgb.XGBClassifier(**params)

In [0]:
%%time
pipeline = train_pipeline(clf=xgb_clf, data=df, ignore_vars=None, ordinal_feats=["Month"], test_size=0.0, 
                             scoring='f1_weighted', grid_search=False, sampling_params=sampling_params, 
                             show_cm=False, include_importance=False, full_train=True)

In [0]:
trained_model = pipeline.named_steps['clf']
preprocessor = pipeline.named_steps['preprocessor']

In [0]:
# Get current timestamp
timestamp = "20241001_144429"#datetime.now().strftime('%Y%m%d_%H%M%S')
# Save the trained pipeline
joblib.dump(trained_model, f'models/xgb_model_{timestamp}.pkl')
joblib.dump(preprocessor, f'models/xgb_preprocessor_{timestamp}.pkl')
print(timestamp)